## Criando Dataframme 

In [0]:
# Criando um DataFrame com dados fixos
dados = [
    ("Grimaldo", "Oliveira", "Brasileira", "Professor", "M", 3000),
    ("Ana", "Santos", "Portuguesa", "Atriz", "F", 4000),
    ("Roberto", "Carlos", "Brasileira", "Analista", "M", 4000),
    ("Maria", "Santanna", "Italiana", "Dentista", "F", 6000),
    ("Jeane", "Andrade", "Inglesa", "Medica", "F", 7000)
]

colunas = ["Primeiro_Nome", "Ultimo_nome", "Nacionalidade", "Trabalho", "Genero", "Salario"]

# Criando o DataFrame no Spark
datafparquet = spark.createDataFrame(dados, colunas)

# Exibindo o conteúdo do DataFrame
datafparquet.show()


+-------------+-----------+-------------+---------+------+-------+
|Primeiro_Nome|Ultimo_nome|Nacionalidade| Trabalho|Genero|Salario|
+-------------+-----------+-------------+---------+------+-------+
|     Grimaldo|   Oliveira|   Brasileira|Professor|     M|   3000|
|          Ana|     Santos|   Portuguesa|    Atriz|     F|   4000|
|      Roberto|     Carlos|   Brasileira| Analista|     M|   4000|
|        Maria|   Santanna|     Italiana| Dentista|     F|   6000|
|        Jeane|    Andrade|      Inglesa|   Medica|     F|   7000|
+-------------+-----------+-------------+---------+------+-------+



## Particionando os dados do arquivo parquet em grupos

In [0]:
datafparquet.write.partitionBy("Nacionalidade", "Salario").format("parquet").save("/FileStore/tables/parquet/pessoal.parquet")

In [0]:
%fs  ls /FileStore/tables/parquet/pessoal.parquet

path,name,size,modificationTime
dbfs:/FileStore/tables/parquet/pessoal.parquet/Nacionalidade=Brasileira/,Nacionalidade=Brasileira/,0,0
dbfs:/FileStore/tables/parquet/pessoal.parquet/Nacionalidade=Inglesa/,Nacionalidade=Inglesa/,0,0
dbfs:/FileStore/tables/parquet/pessoal.parquet/Nacionalidade=Italiana/,Nacionalidade=Italiana/,0,0
dbfs:/FileStore/tables/parquet/pessoal.parquet/Nacionalidade=Portuguesa/,Nacionalidade=Portuguesa/,0,0
dbfs:/FileStore/tables/parquet/pessoal.parquet/_SUCCESS,_SUCCESS,0,1738350222000


## Exibindo os dados do parquet cuja a nacionalidade é brasleira


In [0]:
datafnascional = spark.read.format("parquet").load("/FileStore/tables/parquet/pessoal.parquet/Nacionalidade=Brasileira/")
datafnascional.show(truncate=False)

+-------------+-----------+---------+------+-------+
|Primeiro_Nome|Ultimo_nome|Trabalho |Genero|Salario|
+-------------+-----------+---------+------+-------+
|Grimaldo     |Oliveira   |Professor|M     |3000   |
|Roberto      |Carlos     |Analista |M     |4000   |
+-------------+-----------+---------+------+-------+



## Realizar comando SQL no arquivo particionado


In [0]:
spark.sql("""
    CREATE OR REPLACE TEMPORARY VIEW tb_cidadao 
    USING parquet 
    OPTIONS (path "/FileStore/tables/parquet/pessoal.parquet/Nacionalidade=Brasileira")
""")

spark.sql("SELECT * FROM tb_cidadao WHERE Ultimo_nome = 'Carlos'").show()


+-------------+-----------+--------+------+-------+
|Primeiro_Nome|Ultimo_nome|Trabalho|Genero|Salario|
+-------------+-----------+--------+------+-------+
|      Roberto|     Carlos|Analista|     M|   4000|
+-------------+-----------+--------+------+-------+

